In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import library

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Read data

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')

x_data = train.drop(['Id', 'Cover_Type', 'Soil_Type1', 'Soil_Type7', 'Soil_Type15'], axis=1)
x_test = test.drop(['Id', 'Soil_Type1', 'Soil_Type7', 'Soil_Type15'], axis=1)

y_data = train.Cover_Type

In [ ]:
from sklearn.preprocessing import QuantileTransformer

In [ ]:
transformer = QuantileTransformer(random_state=21, output_distribution='uniform')
x_data = transformer.fit_transform(x_data)
x_test = transformer.transform(x_test)

### Split train and val data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data)

### Train model

In [ ]:
model = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor')
model.fit(x_train, y_train)
y_pred = model.predict(x_val)

acc = accuracy_score(y_val, y_pred)
print(f'validation acc {acc}')
print(f'train acc {accuracy_score(y_train, model.predict(x_train))}')

### Predict test label

In [ ]:
y_test = model.predict(x_test)

### Save prediction and submission

In [ ]:
submission = pd.DataFrame(y_test, columns = ['Cover_Type'], index = test.Id)
submission.to_csv('submission.csv')

In [ ]:
# submission = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
# submission.Cover_Type = y_test
# submission.to_csv('submission.csv', index=False)